## How does everything interact with everthing?!!

In [430]:
import pandas as pd
import numpy as np
import itertools

from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [431]:
df = pd.read_csv('../cleanedData/X.imputed.csv', index_col=0)
y = pd.read_csv('../raw/y.csv')

In [432]:
df.shape

(17290, 20)

# Dropping columns

Things like year built

In [459]:
to_drop = [
    'id',
    'date',
    'zipcode', # overall the model improvess
#    'sqft_basement', # Leave it in, let's see if some manipulations change the effect
#     'yr_renovated'
]

In [436]:
df.drop(columns=to_drop, inplace=True)

In [437]:
df.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,4,2.25,2070,8893,2.0,0,0,4,8,2070,1986,0,47.4388,-122.162,2390,7700
1,5,3.00,2900,6730,1.0,0,0,5,8,1830,1977,0,47.6784,-122.285,2370,6283
2,4,2.50,3770,10893,2.0,0,2,3,11,3770,1997,0,47.5646,-122.129,3710,9685
3,3,3.50,4560,14608,2.0,0,2,3,12,4560,1990,0,47.6995,-122.228,4050,14226
4,3,2.50,2550,5376,2.0,0,0,3,9,2550,2004,0,47.6647,-122.083,2250,4050


## Multiply everything with everything

# To DO change yr_built to house age and yr_renovated to age

In [438]:
len(set(list(itertools.permutations(df.columns,2))))

240

In [439]:
def get_interactions(df, choose=2):
    """
    df --> pd.seriesdataframe
    returns --> pd.dataframe
    
    Given a dataframe, this function takes all of the columns and generates an a new column based on their
    multiplicative interaction.
    
    dependent on itertools and pandas
    """
    columns = itertools.permutations(df.columns,choose)
    interaction = {' X '.join(x): df[x[0]]*df[x[0]] for x in columns}
    return pd.DataFrame.from_dict(interaction)

In [440]:
interaction_df = get_interactions(df)


In [441]:
interaction_df.head()

,bedrooms X bathrooms,bedrooms X sqft_living,bedrooms X sqft_lot,bedrooms X floors,bedrooms X waterfront,bedrooms X view,bedrooms X condition,bedrooms X grade,bedrooms X sqft_above,bedrooms X yr_built,...,sqft_lot15 X waterfront,sqft_lot15 X view,sqft_lot15 X condition,sqft_lot15 X grade,sqft_lot15 X sqft_above,sqft_lot15 X yr_built,sqft_lot15 X yr_renovated,sqft_lot15 X lat,sqft_lot15 X long,sqft_lot15 X sqft_living15
0,16,16,16,16,16,16,16,16,16,16,...,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000
1,25,25,25,25,25,25,25,25,25,25,...,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089
2,16,16,16,16,16,16,16,16,16,16,...,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225
3,9,9,9,9,9,9,9,9,9,9,...,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076
4,9,9,9,9,9,9,9,9,9,9,...,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500


# Removed one of the features to avoid colinearity

In [442]:
interaction_df.iloc[:,:-1]

,bedrooms X bathrooms,bedrooms X sqft_living,bedrooms X sqft_lot,bedrooms X floors,bedrooms X waterfront,bedrooms X view,bedrooms X condition,bedrooms X grade,bedrooms X sqft_above,bedrooms X yr_built,...,sqft_lot15 X floors,sqft_lot15 X waterfront,sqft_lot15 X view,sqft_lot15 X condition,sqft_lot15 X grade,sqft_lot15 X sqft_above,sqft_lot15 X yr_built,sqft_lot15 X yr_renovated,sqft_lot15 X lat,sqft_lot15 X long
0,16,16,16,16,16,16,16,16,16,16,...,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000
1,25,25,25,25,25,25,25,25,25,25,...,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089
2,16,16,16,16,16,16,16,16,16,16,...,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225
3,9,9,9,9,9,9,9,9,9,9,...,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076
4,9,9,9,9,9,9,9,9,9,9,...,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,25,25,25,25,25,25,25,25,25,25,...,108160000,108160000,108160000,108160000,108160000,108160000,108160000,108160000,108160000,108160000
17286,4,4,4,4,4,4,4,4,4,4,...,16104169,16104169,16104169,16104169,16104169,16104169,16104169,16104169,16104169,16104169
17287,9,9,9,9,9,9,9,9,9,9,...,88416409,88416409,88416409,88416409,88416409,88416409,88416409,88416409,88416409,88416409
17288,9,9,9,9,9,9,9,9,9,9,...,11670048784,11670048784,11670048784,11670048784,11670048784,11670048784,11670048784,11670048784,11670048784,11670048784


In [443]:
# Create data frame with original columns plus all interactions
X = pd.concat([df, interaction_df.iloc[:,:-1]], axis=1)
y = pd.read_csv('../raw/y.csv', header=None)
y.columns=['price']


In [444]:
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,sqft_lot15 X floors,sqft_lot15 X waterfront,sqft_lot15 X view,sqft_lot15 X condition,sqft_lot15 X grade,sqft_lot15 X sqft_above,sqft_lot15 X yr_built,sqft_lot15 X yr_renovated,sqft_lot15 X lat,sqft_lot15 X long
0,4,2.25,2070,8893,2.0,0,0,4,8,2070,...,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000
1,5,3.00,2900,6730,1.0,0,0,5,8,1830,...,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089
2,4,2.50,3770,10893,2.0,0,2,3,11,3770,...,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225
3,3,3.50,4560,14608,2.0,0,2,3,12,4560,...,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076
4,3,2.50,2550,5376,2.0,0,0,3,9,2550,...,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500


In [445]:
# Year renovated and sqft_basement are currently in the dataframe. Meaning we have their interations too
X.columns.to_list()

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'bedrooms X bathrooms',
 'bedrooms X sqft_living',
 'bedrooms X sqft_lot',
 'bedrooms X floors',
 'bedrooms X waterfront',
 'bedrooms X view',
 'bedrooms X condition',
 'bedrooms X grade',
 'bedrooms X sqft_above',
 'bedrooms X yr_built',
 'bedrooms X yr_renovated',
 'bedrooms X lat',
 'bedrooms X long',
 'bedrooms X sqft_living15',
 'bedrooms X sqft_lot15',
 'bathrooms X bedrooms',
 'bathrooms X sqft_living',
 'bathrooms X sqft_lot',
 'bathrooms X floors',
 'bathrooms X waterfront',
 'bathrooms X view',
 'bathrooms X condition',
 'bathrooms X grade',
 'bathrooms X sqft_above',
 'bathrooms X yr_built',
 'bathrooms X yr_renovated',
 'bathrooms X lat',
 'bathrooms X long',
 'bathrooms X sqft_living15',
 'bathrooms X sqft_lot15',
 'sqft_living X bedrooms',
 'sqft_living X bathrooms

In [446]:
# Split the data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y,               # Pass in our X and y
                                                    random_state=42,    # Abritary select a random_state 
                                                    test_size=.2        # Split test size to be 20% of full data.
                                                   )

In [447]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_hat =lr.predict(X_test)
RMSE = mean_squared_error(y_test, y_hat)
RMSE**0.5

181224.05535060973

In [448]:
from sklearn.model_selection import cross_val_score
lr = LinearRegression()
(cross_val_score(estimator=lr, X=X_train, y=y_train, cv = 10))

array([0.62262089, 0.71305742, 0.78810355, 0.69977926, 0.75644202,
       0.76682247, 0.73707596, 0.75913398, 0.73205334, 0.75106641])

## Now that we have a model with all of the interaction features, we can concatenate it with the all of our geospatial features time series features

In [449]:
# X_full_model_one has all of our geospatial and time data in it
X_full_model_one = pd.read_csv('../raw/X_full_model_one.csv', index_col=0)

In [450]:
# first 21 features are repeated in our X dataframe so we can drop them
list(enumerate(list(X_full_model_one.columns)))

[(0, 'id'),
 (1, 'date'),
 (2, 'bedrooms'),
 (3, 'bathrooms'),
 (4, 'sqft_living'),
 (5, 'sqft_lot'),
 (6, 'floors'),
 (7, 'waterfront'),
 (8, 'view'),
 (9, 'condition'),
 (10, 'grade'),
 (11, 'sqft_above'),
 (12, 'sqft_basement'),
 (13, 'yr_built'),
 (14, 'yr_renovated'),
 (15, 'zipcode'),
 (16, 'lat'),
 (17, 'long'),
 (18, 'sqft_living15'),
 (19, 'sqft_lot15'),
 (20, 'c22uz'),
 (21, 'c22vm'),
 (22, 'c22vn'),
 (23, 'c22vp'),
 (24, 'c22vr'),
 (25, 'c22vs'),
 (26, 'c22vt'),
 (27, 'c22vu'),
 (28, 'c22vv'),
 (29, 'c22vw'),
 (30, 'c22vz'),
 (31, 'c22yj'),
 (32, 'c22yp'),
 (33, 'c22yq'),
 (34, 'c22yr'),
 (35, 'c22yw'),
 (36, 'c22yx'),
 (37, 'c22yy'),
 (38, 'c22yz'),
 (39, 'c22zn'),
 (40, 'c22zp'),
 (41, 'c22zq'),
 (42, 'c22zr'),
 (43, 'c22zx'),
 (44, 'c22zy'),
 (45, 'c22zz'),
 (46, 'c23hb'),
 (47, 'c23hc'),
 (48, 'c23hf'),
 (49, 'c23hg'),
 (50, 'c23hr'),
 (51, 'c23hu'),
 (52, 'c23hv'),
 (53, 'c23hw'),
 (54, 'c23hx'),
 (55, 'c23hy'),
 (56, 'c23hz'),
 (57, 'c23j0'),
 (58, 'c23j1'),
 (59, 'c23

In [451]:
# supset to remove the first 21 columns which overlap with our X column to avoid double ups
X_full_model_one = X_full_model_one.iloc[:,21:]

In [452]:
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,sqft_lot15 X floors,sqft_lot15 X waterfront,sqft_lot15 X view,sqft_lot15 X condition,sqft_lot15 X grade,sqft_lot15 X sqft_above,sqft_lot15 X yr_built,sqft_lot15 X yr_renovated,sqft_lot15 X lat,sqft_lot15 X long
0,4,2.25,2070,8893,2.0,0,0,4,8,2070,...,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000
1,5,3.00,2900,6730,1.0,0,0,5,8,1830,...,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089
2,4,2.50,3770,10893,2.0,0,2,3,11,3770,...,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225
3,3,3.50,4560,14608,2.0,0,2,3,12,4560,...,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076
4,3,2.50,2550,5376,2.0,0,0,3,9,2550,...,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500


In [453]:
# Create our new X
X = pd.concat([X_full_model_one, X], axis=1)

In [454]:
# 571 features!!
X.head()

,c22vm,c22vn,c22vp,c22vr,c22vs,c22vt,c22vu,c22vv,c22vw,c22vz,...,sqft_lot15 X floors,sqft_lot15 X waterfront,sqft_lot15 X view,sqft_lot15 X condition,sqft_lot15 X grade,sqft_lot15 X sqft_above,sqft_lot15 X yr_built,sqft_lot15 X yr_renovated,sqft_lot15 X lat,sqft_lot15 X long
0,0,0,0,0,0,0,0,0,0,0,...,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000,59290000
1,0,0,0,0,0,0,0,0,0,0,...,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089,39476089
2,0,0,0,0,0,0,0,0,0,0,...,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225,93799225
3,0,0,0,0,0,0,0,0,0,0,...,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076,202379076
4,0,0,0,0,0,0,0,0,0,0,...,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500,16402500


In [455]:
# Split the data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y,               # Pass in our X and y
                                                    random_state=42,    # Abritary select a random_state 
                                                    test_size=.2        # Split test size to be 20% of full data.
                                                   )

In [456]:
y_train.shape, X_train.shape

((13832, 1), (13832, 503))

In [457]:
lr = LinearRegression()
lr.fit(X_train, y_train.price)
y_hat =lr.predict(X_test)
RMSE = mean_squared_error(y_test, y_hat)

In [458]:
RMSE**0.5

136340.7514629376

In [318]:
y_hat

array([443921.5506199 , 942128.33173388, 322454.7713455 , ...,
       167687.85767967, 730984.12396872, 340504.2551623 ])

In [321]:
from scipy.stats import sem

In [325]:
from sklearn.model_selection import cross_val_score
lr = LinearRegression()
cross_val = (cross_val_score(estimator=lr, X=X, y=y, cv = 10))
cross_val_mean = round(np.mean(cross_val),2)
cross_val_std = round(np.std(cross_val),2)

standard_error = round(sem(cross_val), 2)

print(f"The mean cross value R squared value is {cross_val_mean}, with a standard deviation of {cross_val_std} \
with a standard error of {standard_error}")

The mean cross value R squared value is 0.84, with a standard deviation of 0.03 with a standard error of 0.01


## This is a very strong model! 

Let's repeat the process for our holdout data set!

In [328]:
holdout = pd.read_csv('../raw/kc_holdout_data.csv', index_col=0)

to_drop = [
    'date', # handled as dummy columns/seasons
    'id', # randomly generated
    'zipcode', # handled as dummy columns
    'yr_renovated',
]
holdout.drop(columns=to_drop, inplace = True)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,lat,long,sqft_living15,sqft_lot15
0,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,47.7089,-122.241,2020,10918
1,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,47.7089,-122.241,2020,10918
2,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,47.5472,-121.998,1470,1576
3,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,47.7427,-122.071,1160,10565
4,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,47.4863,-122.140,2830,7916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,47.6993,-122.346,1530,1509
4319,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,47.5107,-122.362,1830,7200
4320,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,47.5944,-122.299,1020,2007
4321,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,47.5345,-122.069,1410,1287


In [ ]:
holdout = pd.read_csv('../raw/kc_holdout_data.csv', index_col=0)

# create interaction dataframe

# drop columns we are not interested in
holdout.drop(columns=to_drop, inplace=True)

# Run the holdout dataframe through my function

holdout_interactions = get_interactions(holdout)

holdout_interactions.shape

# # test the model at scale
# holdout = pd.read_csv('../raw/kc_holdout_data.csv')
# # Get zipcode dummies 
# zipcodes_dummies = pd.get_dummies(holdout.zipcode)

to_drop = [
    'date', # handled as dummy columns/seasons
    'id', # randomly generated
    'zipcode', # handled as dummy columns
    'yr_renovated',

]
# get geo

### So the best we can do is 150K range, that has no feature normalization at all. How do we improve this?

holdoutZipcodes = pd.read_csv('../cleanedData/holdoutZipcodes.csv', index_col=0)
holdoutGeo = pd.read_csv('../cleanedData/holdoutGeo.csv', index_col=0)
holdoutSeason = pd.read_csv('../cleanedData/holdoutSeason.csv', index_col=0)
X_holdout = pd.read_csv('../raw/kc_holdout_data.csv', index_col=0)



holdout_df = pd.concat([X_holdout, holdoutGeo,holdoutZipcodes, holdoutSeason], axis=1)
holdout_df = holdout_df.drop(columns=to_drop)

hold_out_full = pd.concat([holdout_df,holdout_interactions], axis=1)

# Split the data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y,               # Pass in our X and y
                                                    random_state=42,    # Abritary select a random_state 
                                                    test_size=.2        # Split test size to be 20% of full data.
                                                   )

X_train

lr = LinearRegression()
lr.fit(X, y)
y_hat =lr.predict(hold_out_full)
# RMSE = mean_squared_error(y, y_hat)

pd.DataFrame(y_hat).to_csv('../cleanedData/modeltwo.csv')

from sklearn.model_selection import cross_val_score
lr = LinearRegression()
(cross_val_score(estimator=lr, X=X_train, y=y_train, cv = 10))

pd.read_csv('../cleanedData/modeltwo.csv')